In [2]:
# 1. import libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re

In [3]:
# 2. find url and store it in a variable
url = "https://www.billboard.com/charts/hot-100/"

In [4]:
# 3. download html with a get request. Use the function request.get() and store the output in response
response = requests.get(url)
# 200 status code means OK! response.status_code
print(response.status_code)

200


In [5]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.text, 'html.parser')
# 4.2. check that the html code looks like it should
#print(soup.prettify())

In [6]:
song = [elem.get_text(" ", strip=True) for elem in soup.select("li h3.c-title")]
song

['Flowers',
 'Kill Bill',
 'Last Night',
 'Unholy',
 "Creepin'",
 'Cuff It',
 'Anti-Hero',
 'Die For You',
 "I'm Good (Blue)",
 'As It Was',
 'Rich Flex',
 'Just Wanna Rock',
 'Thought You Should Know',
 "Boy's A Liar, Pt. 2",
 'Made You Look',
 'Under The Influence',
 'Golden Hour',
 'You Proof',
 'Rock And A Hard Place',
 'Something In The Orange',
 'Thank God',
 'Bad Habit',
 'Going, Going, Gone',
 'Escapism',
 'Superhero (Heroes & Villains)',
 'Lavender Haze',
 'Calm Down',
 'Players',
 'Shirt',
 'Sure Thing',
 'The Kind Of Love We Make',
 'Heart Like A Truck',
 'Bzrp Music Sessions, Vol. 53',
 'Snooze',
 'Until I Found You',
 'Wait For U',
 'I Like You (A Happier Song)',
 'I Wrote The Book',
 'Bebe Dame',
 'Tomorrow 2',
 'About Damn Time',
 'Wait In The Truck',
 'She Had Me At Heads Carolina',
 'Nobody Gets Me',
 'What My World Spins Around',
 'Unstoppable',
 'One Thing At A Time',
 'Whiskey On You',
 'Dawns',
 'Love Again',
 'Everything I Love',
 'Lift Me Up',
 'Que Vuelvas',
 'L

In [7]:
artist = [elem.get_text(" ", strip=True) for elem in soup.select("li span.c-label.a-no-trucate")]
artist

['Miley Cyrus',
 'SZA',
 'Morgan Wallen',
 'Sam Smith & Kim Petras',
 'Metro Boomin, The Weeknd & 21 Savage',
 'Beyonce',
 'Taylor Swift',
 'The Weeknd',
 'David Guetta & Bebe Rexha',
 'Harry Styles',
 'Drake & 21 Savage',
 'Lil Uzi Vert',
 'Morgan Wallen',
 'PinkPantheress & Ice Spice',
 'Meghan Trainor',
 'Chris Brown',
 'JVKE',
 'Morgan Wallen',
 'Bailey Zimmerman',
 'Zach Bryan',
 'Kane Brown With Katelyn Brown',
 'Steve Lacy',
 'Luke Combs',
 'RAYE Featuring 070 Shake',
 'Metro Boomin, Future & Chris Brown',
 'Taylor Swift',
 'Rema & Selena Gomez',
 'Coi Leray',
 'SZA',
 'Miguel',
 'Luke Combs',
 'Lainey Wilson',
 'Bizarrap & Shakira',
 'SZA',
 'Stephen Sanchez',
 'Future Featuring Drake & Tems',
 'Post Malone Featuring Doja Cat',
 'Morgan Wallen',
 'Fuerza Regida X Grupo Frontera',
 'GloRilla & Cardi B',
 'Lizzo',
 'HARDY Featuring Lainey Wilson',
 'Cole Swindell',
 'SZA',
 'Jordan Davis',
 'Sia',
 'Morgan Wallen',
 'Nate Smith',
 'Zach Bryan Featuring Maggie Rogers',
 'The Kid L

In [8]:
hot100_df = pd.DataFrame({"artist":artist, "song":song})
hot100_df

,artist,song
0,Miley Cyrus,Flowers
1,SZA,Kill Bill
2,Morgan Wallen,Last Night
3,Sam Smith & Kim Petras,Unholy
4,"Metro Boomin, The Weeknd & 21 Savage",Creepin'
...,...,...
95,Yandel & Feid,Yandel 150
96,Mac DeMarco,Heart To Heart
97,Jordan Davis,Next Thing You Know
98,P!nk,Never Gonna Not Dance Again


In [9]:
hot100_df['song'] = hot100_df['song'].map(str.lower) # converting the entries to lower
hot100_df['artist'] = hot100_df['artist'].map(str.lower) # converting the entries to lower
hot100_df

hot100_df.to_csv("artistsong.csv", index=False)

In [10]:
import pyinputplus as pyip   #to insert predefined user input functions
import random

user_song = str(input("Choose your song: "))    
rec_song = hot100_df["song"].sample().to_string(index=False)     #randomly reccomend another song with sample, .to_string avoit to put the index too
rec_artist = hot100_df.loc[hot100_df["song"] == rec_song, "artist"].to_string(index=False)   #associate the artist to the random reccomended song

x_artist = hot100_df.loc[hot100_df['song'].map(str.lower)  == user_song.lower(), 'artist'] # artist corresponding to song x
artist_question = pyip.inputYesNo('Did you mean ' + user_song.lower().title() + ' by ' + x_artist.to_string(index=False).title() + '?')  #this asks if the user mean an identical song but not in the list, if neither Y or N are given, it asks again

if artist_question == "yes":
    if user_song.lower() in hot100_df.values and user_song.lower() != rec_song:         #.lower() exclude typos from capital/lower letter "and user_song.lower() != rec_song" exclude to randomize including the same song
        print(f"The song you have chosen is " + user_song.title() + ". Why don't you try to listen to " + rec_song.title() + " by " + rec_artist.title())     #.title() gives results with capital first letter
    else:
        print("Your song is not in Billboard 100 Hot")
else:
    print("Your song is not in Billboard 100 Hot")
    


Choose your song: flowers
Did you mean Flowers by Miley Cyrus?y
The song you have chosen is Flowers. Why don't you try to listen to Bloody Mary by Lady Gaga


# Playlist creation from Spotify API

In [11]:
import config
import pprint
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [12]:
# Rock classics playlist link: https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=013db49871054a73
# Rock classics playlist ID: 37i9dQZF1DWXRqgorJj26U

playlist_rock = sp.user_playlist_tracks("spotify", "37i9dQZF1DWXRqgorJj26U")


In [13]:
print(list(playlist_rock.keys())) # Let's look at items and total:
print("Total number of songs in the playlist: ",playlist_rock["total"]) #  Let's check items:
len(playlist_rock["items"]) # It is limited to 100 tracks, we will have to fix it:

['href', 'items', 'limit', 'next', 'offset', 'previous', 'total']
Total number of songs in the playlist:  202


100

In [14]:
playlist_rock["items"][0]["track"]["id"]


'1xsYj84j7hUDDnTTerGWlH'

In [15]:
playlist_rock_id = "37i9dQZF1DWXRqgorJj26U"

def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [16]:
rock_tracks = get_playlist_tracks("spotify", "37i9dQZF1DWXRqgorJj26U")

In [17]:
#extract the song id from playlist
list_of_audio_features=[]
for item in range(0,100):
    #print (tracks[item]["track"]["id"])
    list_of_audio_features.append(sp.audio_features(rock_tracks[item]["track"]["id"])[0])

In [18]:
df=pd.DataFrame(list_of_audio_features)    
df=df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.392,0.476,-7.583,0.0249,0.397000,0.000163,0.3090,0.234,78.031,1xsYj84j7hUDDnTTerGWlH,267597
1,0.548,0.691,-13.553,0.0369,0.015000,0.106000,0.1440,0.667,137.145,4KfSdst7rW39C0sfhArdrz,261933
2,0.413,0.881,-5.541,0.0367,0.000060,0.000308,0.0805,0.364,158.066,5UWwZ5lm5PKu6eKsHAGxOk,250547
3,0.532,0.887,-3.757,0.0335,0.077800,0.000214,0.2940,0.795,122.511,37ZJ0p5Jm13JPevGcx4SkF,249293
4,0.545,0.640,-9.937,0.2240,0.002490,0.030400,0.1960,0.606,80.544,70YvYr2hGlS01bKRIho1HM,230480
...,...,...,...,...,...,...,...,...,...,...,...
95,0.573,0.939,-6.441,0.0808,0.493000,0.000000,0.0994,0.963,137.382,29SyMC0plk6qw8NMF7lfRL,134253
96,0.468,0.607,-11.367,0.0336,0.072000,0.000852,0.2250,0.886,98.429,3MODES4TNtygekLl146Dxd,263333
97,0.446,0.952,-5.321,0.0523,0.090100,0.085700,0.1120,0.624,125.303,7snQQk1zcKl8gZ92AnueZW,356067
98,0.632,0.586,-11.333,0.0284,0.098300,0.581000,0.0535,0.890,114.243,5SAUIWdZ04OxYfJFDchC7S,342173


In [19]:
# defines the variable 'tracks' and assigns the different playlists
playlist_ids = ["27gN69ebwiJRtXEboL12Ih", "37i9dQZF1DX4pUKG1kS0Ac", "37i9dQZF1DXbITWG1ZJKYt", 
                "37i9dQZF1DX8a1tdzq5tbM", "37i9dQZF1DX2OwDeeyTh0V", "37i9dQZF1DXd9rSDyQguIk",
                "37i9dQZEVXbMDoHDwVN2tF", "37i9dQZF1DX6R7QUWePReA", "37i9dQZF1DWWM6GBnxtToT", 
                "37i9dQZF1DX4UtSsGT1Sbe","37i9dQZF1DXbTxeAdrVG2l", "37i9dQZF1DWTJ7xPn4vNaz", 
                "37i9dQZF1DX5Ejj0EkURtP", "37i9dQZF1DX4o1oenSJRJd", "37i9dQZF1DWSf2RDTDayIx",
               "37i9dQZF1DX9XIFQuFvzM4", "37i9dQZF1DX4fpCWaHOned", "37i9dQZF1DX5Ejj0EkURtP"]

tracks = []

for x in playlist_ids:
    tracks += get_playlist_tracks("spotify", x)
# tracks = get_playlist_tracks("spotify", "27gN69ebwiJRtXEboL12Ih") + get_playlist_tracks("spotify", "37i9dQZF1DX4pUKG1kS0Ac")


In [20]:
len(tracks)

2176

In [21]:
#extract the song id from playlist
list_of_audio_features=[]
for item in range(0,(len(tracks))):
    #print (tracks[item]["track"]["id"])
    list_of_audio_features.append(sp.audio_features(tracks[item]["track"]["id"])[0])

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [ ]:
df=pd.DataFrame(list_of_audio_features)    
df=df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

df

In [ ]:
df.to_csv("hot100_billboard.csv", index=False)